# Get a single face from a video

In [10]:
from fastai.vision import *
from kgl_deepfake.nb_00 import *
from kgl_deepfake.nb_01 import *
from IPython.display import HTML
import cv2
import pandas as pd
from facenet_pytorch import MTCNN
from nbdev.export import *

In [2]:
SOURCE = Path('../data/train_sample_videos/')

In [7]:
f = get_files(SOURCE, extensions=['.json'])[0]
annots = pd.read_json(f).T
annots.reset_index(inplace=True)
annots.rename({'index':'fname'}, axis=1, inplace=True)
annots.head()

,fname,label,split,original
0,aagfhgtpmv.mp4,FAKE,train,vudstovrck.mp4
1,aapnvogymq.mp4,FAKE,train,jdubbvfswz.mp4
2,abarnvbtwb.mp4,REAL,train,None
3,abofeumbvv.mp4,FAKE,train,atvmxvwyns.mp4
4,abqwwspghj.mp4,FAKE,train,qzimuostzz.mp4


In [9]:
fn = SOURCE/annots.fname[0]
fn

PosixPath('../data/train_sample_videos/aagfhgtpmv.mp4')

In [14]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
detector = MTCNN(device=device, post_process=False)

In [38]:
resize = .5

In [60]:
def get_first_face(detector, fn, resize=.5):
    '''
    Returns the first detected face from a video
    '''
    v_cap = cv2.VideoCapture(str(fn))
    v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    iframe, face = None, None
    for j in range(v_len):
        _ = v_cap.grab()
        success, frame = v_cap.retrieve()
        if not success: continue
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = PIL.Image.fromarray(frame)
        if resize is not None: frame = frame.resize([int(d * resize) for d in frame.size])
        faces = detector(frame)
        if faces is not None: 
            iframe, face = i, faces[0]
            break
        v_cap.release()
    return iframe, face

### Videos in which not a single face is detected by MTCNN

In [45]:
iframes = []
for i in progress_bar(range(annots.shape[0])):
    r = annots.iloc[i]
    iframe, face = get_first_face(detector, SOURCE/r.fname)
    iframes.append(iframe)

In [49]:
len(iframes), len([o for o in iframes if o is not None])

(400, 380)

In [52]:
iframes_noface = [i for i, iframe in enumerate(iframes) if iframe is None]

In [58]:
fnames_noface = [SOURCE/o for o in annots.iloc[iframes_noface].fname]
labels = [f'{o.fname} {o.label}' for i, o in annots.iloc[iframes_noface].iterrows()]

In [59]:
HTML(html_vids(fnames_noface, titles=labels))